# Appendix

**Proof that $F_x ,F_z$ and $T_y$ are the only three components of wrench affecting torque at Joint 3 when all other joints are fixed at $0 rad$.**

In [12]:
import sympy as sp
import numpy as np

# Step 1: Define symbolic variable for theta3 (only variable)
theta3 = sp.symbols('theta3')

# Other joint angles are zero
theta1 = theta2 = theta4 = theta5 = theta6 = 0

# Step 2: Define the numerical DH parameters using sympy
dh_params_symbolic = [
    (0, -sp.pi/2, 0.26, theta1),       # Link 1
    (0.48, 0, 0, theta2 - sp.pi/2),    # Link 2
    (0, sp.pi/2, 0, theta3 + sp.pi/2), # Link 3 (only theta3 is variable)
    (0, -sp.pi/2, 0.7, theta4),        # Link 4
    (0, sp.pi/2, 0, theta5),           # Link 5
    (0, 0, 0.1735, theta6)             # Link 6
]

# Step 3: Define DH transformation matrix function using sympy
def dh_transform_symbolic(a, alpha, d, theta):
    return sp.Matrix([
        [sp.cos(theta), -sp.sin(theta) * sp.cos(alpha), sp.sin(theta) * sp.sin(alpha), a * sp.cos(theta)],
        [sp.sin(theta), sp.cos(theta) * sp.cos(alpha), -sp.cos(theta) * sp.sin(alpha), a * sp.sin(theta)],
        [0, sp.sin(alpha), sp.cos(alpha), d],
        [0, 0, 0, 1]
    ])

# Step 4: Compute forward kinematics with this specific configuration
def compute_forward_kinematics_symbolic(dh_params):
    T = sp.eye(4)
    transformations = [T]
    
    for a, alpha, d, theta in dh_params:
        T_i = dh_transform_symbolic(a, alpha, d, theta)
        T = T * T_i
        transformations.append(T)
    
    T_0_n = transformations[-1]
    return transformations, T_0_n

# Compute transformations with only theta3 being non-zero
transformations, T_0_n = compute_forward_kinematics_symbolic(dh_params_symbolic)

# Step 5: Compute Jacobian symbolically
def compute_geometric_jacobian_symbolic(transformations):
    num_joints = len(transformations) - 1  # Exclude base frame
    jacobian_base = sp.zeros(6, num_joints)
    
    T_0_n = transformations[-1]
    o_n = T_0_n[:3, 3]

    for i in range(num_joints):
        T_0_i = transformations[i]
        o_i = T_0_i[:3, 3]
        z_i = T_0_i[:3, 2]

        jacobian_base[:3, i] = z_i.cross(o_n - o_i)  # Linear velocity part
        jacobian_base[3:, i] = z_i  # Angular velocity part

    return jacobian_base

# Compute the Jacobian for this configuration
jacobian = compute_geometric_jacobian_symbolic(transformations)

# Step 6: Isolate the Jacobian column for joint 3 (index 2)
jacobian_joint3 = jacobian[:, 2]

# Display the symbolic Jacobian column for joint 3
sp.pprint(jacobian_joint3)


⎡0.8735⋅cos(θ₃) ⎤
⎢               ⎥
⎢       0       ⎥
⎢               ⎥
⎢-0.8735⋅sin(θ₃)⎥
⎢               ⎥
⎢       0       ⎥
⎢               ⎥
⎢       1       ⎥
⎢               ⎥
⎣       0       ⎦


The third column of the jacobain matrix is $[0.8735 cos(\theta_3), 0, -0.8735 cos(\theta_3), 0, 1, 0]^T$. 
 In a configuration where all other joints are fixed at $0 rad$, by computing the Jacobain
symbolically, it can be seen that the three indices of the third column of the jacobain
matrix that are non-zero correspond to F_x ,F_z and T_y components of the wrench.